In [2]:
# | default_exp trainer.config

In [3]:
# | export
from torch_snippets.registry import parse
from torch_snippets import store_attr, ifnone
import inspect as inspect_builtin

In [61]:
# | export
class DeepLearningConfig:
    """
    A configuration class for deep learning models.

    This class provides methods to access and manipulate configuration settings.

    Attributes:
        input_variables (list): List of input variables defined in the class constructor.

    Methods:
        keys(): Returns the list of input variables.
        __getitem__(key): Returns the value of the specified key.
        __contains__(key): Checks if the specified key is present in the input variables.
        from_ini_file(filepath, config_root=None): Creates an instance of the class from an INI file.
        __repr__(): Returns a string representation of the class.

    Example usage:
        config = DeepLearningConfig()
        config.from_ini_file('config.ini')
        print(config.keys())
        print(config['learning_rate'])
    """

    def keys(self):
        if not hasattr(self, "input_variables"):
            self.input_variables = inspect_builtin.signature(
                self.__init__
            ).parameters.keys()
        return self.input_variables

    def __getitem__(self, key):
        return getattr(self, key)

    def __contains__(self, key):
        return key in self.input_variables

    @classmethod
    def from_ini_file(cls, filepath, *, config_root=None):
        config = parse(filepath)
        config_root = ifnone(config_root, getattr(cls, "config_root"))
        if config_root is not None:
            for _root in config_root.split("."):
                config = config[_root]
        return cls(**config)

    def __repr__(self):
        return f"{self.__class__.__name__}:\n" + str({**self})

In [62]:
from torch_snippets.registry import parse_string
from torch_snippets.torch_loader import *
from torch_snippets import writelines

MNIST(
  (model): Sequential(
    (0): Linear(in_features=768, out_features=256, bias=True)
    (1): Sequential(
      (0): Linear(in_features=256, out_features=256, bias=True)
      (1): ReLU()
    )
    (2): Sequential(
      (0): Linear(in_features=256, out_features=256, bias=True)
      (1): ReLU()
    )
    (3): Linear(in_features=256, out_features=10, bias=True)
  )
)


In [ ]:
config_str = """
[META]
experiment = mnist.v1
description = Training MLP with 
    mnist data on 10k images only
    using huggingface trainer and 
    cosine annealing

[ModelConfig]
n_layers = 3
n_hidden = 256
n_classes = 10

[DatasetConfig]
root = /home/datasets/mnist
train = ${root}/train
val = ${root}/val
train_subset = 10000
val_subest = ${train_subset}//10

[TrainingConfig]
max_steps = ${DatasetConfig.train_subset} * 5
learning_rate = 3e-4
output_dir = ./results/${META.experiment}
per_device_train_batch_size = 256
per_device_eval_batch_size = ${per_device_train_batch_size}
evaluation_strategy = "steps"
eval_steps = 500
logging_strategy = ${evaluation_strategy}
logging_steps = ${eval_steps}//100
save_strategy = ${evaluation_strategy}
save_steps = ${eval_steps}
save_total_limit = 1
seed = 1234
label_names = ['targets']
lr_scheduler_type = cosine
""".strip()

config = parse_string(config_str)


class MNIST(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(768, config.n_hidden),
            *[
                nn.Sequential(nn.Linear(config.n_hidden, config.n_hidden), nn.ReLU())
                for _ in range(config.n_layers - 1)
            ],
            nn.Linear(config.n_hidden, config.n_classes)
        )

    def forward(self, images):
        ...


model = MNIST(config.ModelConfig)
print(model)

If needed, configs can be unpacked like a dictionary too

In [63]:
class MNIST(nn.Module):
    """
    A PyTorch module for a multi-layer perceptron (MLP) model for MNIST classification.

    Args:
        n_hidden (int): The number of hidden units in each hidden layer.
        n_classes (int): The number of output classes.
        n_layers (int): The number of hidden layers in the model.

    Attributes:
        model (nn.Sequential): The sequential model that represents the MLP.

    """

    def __init__(self, *, n_hidden, n_classes, n_layers):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(768, n_hidden),
            *[
                nn.Sequential(nn.Linear(n_hidden, n_hidden), nn.ReLU())
                for _ in range(n_layers - 1)
            ],
            nn.Linear(n_hidden, n_classes)
        )

    def forward(self, images):
        ...


model = MNIST(**config.ModelConfig)
print(model)

MNIST(
  (model): Sequential(
    (0): Linear(in_features=768, out_features=256, bias=True)
    (1): Sequential(
      (0): Linear(in_features=256, out_features=256, bias=True)
      (1): ReLU()
    )
    (2): Sequential(
      (0): Linear(in_features=256, out_features=256, bias=True)
      (1): ReLU()
    )
    (3): Linear(in_features=256, out_features=10, bias=True)
  )
)


In [71]:
# | export
class GenericConfig(DeepLearningConfig):
    def __init__(self, **kwargs):
        super().__init__()
        for k, v in kwargs.items():
            setattr(self, k, v)
        self.input_variables = kwargs.keys()

    @classmethod
    def from_ini_file(cls, filepath, *, config_root):
        config = parse(filepath)
        for _root in config_root.split("."):
            config = config[_root]
        # convert string type to list type
        return cls(**config)

GenericConfig is a special class that can have attributes solely based on the config file, i.e., when we are unsure what are the arguments in the config going to be

In [72]:
writelines(config_str.split("\n"), "/tmp/tmp.ini", "w")
training_config = GenericConfig.from_ini_file(
    "/tmp/tmp.ini", config_root="TrainingConfig"
)


def train(**kwargs):
    for k, v in kwargs.items():
        print(k, v)


train(**training_config)

max_steps 50000
learning_rate 0.00030000000000000003
output_dir ./results/mnist.v1
per_device_train_batch_size 256
per_device_eval_batch_size 256
evaluation_strategy steps
eval_steps 500
logging_strategy steps
logging_steps 5
save_strategy steps
save_steps 500
save_total_limit 1
seed 1234
label_names ['targets']
lr_scheduler_type cosine
